In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

In [3]:
# Load data
df = pd.read_csv("../datasets/vct_combined.csv")  # replace with your file path
df = df.drop(columns=['Unnamed: 0'])

# Filter out columns you don't want
exclude_cols = ['Region', 'Team', 'Result']
X = df.drop(columns=exclude_cols)
y = df['Result'].values

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [4]:
df

,Region,Team,Pistol Rounds Won,First Kills,KAST,Clutches,Eco,Semi-Eco,Half-Buy,Full-Buy,...,+1 Opp,-1 Opp,0 Opp,+2 Opp,-2 Opp,-3 Opp,+3 Opp,+4 Opp,-4 Opp,Result
0,AMERICAS,EG,0.50,23,0.67,0.068966,0.00,0.25,0.33,0.52,...,0.8650,0.2800,0.6050,0.916667,0.046667,0.0,1.0,1.0,0.0,0
1,AMERICAS,LOUD,0.50,26,0.71,0.173913,0.00,0.00,0.43,0.68,...,0.7200,0.1350,0.3950,0.953333,0.083333,0.0,1.0,1.0,0.0,1
2,AMERICAS,MIBR,0.25,11,0.59,0.040000,0.00,0.00,0.25,0.53,...,0.7925,0.3000,0.6750,0.953333,0.166667,0.0,1.0,1.0,0.0,0
3,AMERICAS,100T,0.75,28,0.82,0.200000,0.00,0.00,0.44,0.76,...,0.7000,0.2075,0.3250,0.833333,0.046667,0.0,0.5,0.0,0.0,1
4,AMERICAS,2G,0.50,20,0.67,0.071429,0.00,0.00,0.38,0.37,...,0.9450,0.3450,0.9000,0.850000,0.110000,0.0,1.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,APAC,PRX,0.50,20,0.68,0.173913,0.00,0.67,0.63,0.61,...,0.5425,0.0250,0.1500,0.890000,0.000000,0.0,1.0,1.0,0.0,1
380,APAC,RRQ,0.30,41,0.67,0.196721,0.00,0.44,0.36,0.55,...,0.7500,0.1625,0.3800,1.000000,0.073333,0.0,1.0,1.0,0.0,1
381,APAC,PRX,0.70,61,0.77,0.153846,0.00,0.20,0.61,0.57,...,0.8375,0.2500,0.6200,0.926667,0.000000,0.0,1.0,0.0,0.0,0
382,APAC,GEN,0.50,41,0.68,0.196429,0.33,0.25,0.52,0.49,...,0.7500,0.1250,0.5225,0.973333,0.066667,0.0,1.0,1.0,0.0,0


In [5]:
class ValorantDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = ValorantDataset(X_train, y_train)
test_dataset = ValorantDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [6]:
import torch.nn as nn

class ValorantNet(nn.Module):
    def __init__(self, input_size):
        super(ValorantNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [7]:
model = ValorantNet(input_size=X.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_X, batch_y in train_loader:
        outputs = model(batch_X).squeeze()
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

Epoch 1/30, Loss: 12.8646
Epoch 2/30, Loss: 9.8923
Epoch 3/30, Loss: 6.3241
Epoch 4/30, Loss: 4.0304
Epoch 5/30, Loss: 3.1425
Epoch 6/30, Loss: 2.5879
Epoch 7/30, Loss: 2.1620
Epoch 8/30, Loss: 2.0608
Epoch 9/30, Loss: 1.6468
Epoch 10/30, Loss: 1.6109
Epoch 11/30, Loss: 1.1982
Epoch 12/30, Loss: 1.2088
Epoch 13/30, Loss: 0.8945
Epoch 14/30, Loss: 1.0188
Epoch 15/30, Loss: 1.0985
Epoch 16/30, Loss: 0.6658
Epoch 17/30, Loss: 0.7581
Epoch 18/30, Loss: 0.5011
Epoch 19/30, Loss: 0.5017
Epoch 20/30, Loss: 0.3991
Epoch 21/30, Loss: 0.3903
Epoch 22/30, Loss: 0.4216
Epoch 23/30, Loss: 0.2896
Epoch 24/30, Loss: 0.4177
Epoch 25/30, Loss: 0.1897
Epoch 26/30, Loss: 0.2651
Epoch 27/30, Loss: 0.2109
Epoch 28/30, Loss: 0.2201
Epoch 29/30, Loss: 0.1692
Epoch 30/30, Loss: 0.1292


In [8]:
from sklearn.metrics import accuracy_score

model.eval()
all_preds = []

with torch.no_grad():
    for batch_X, _ in test_loader:
        outputs = model(batch_X).squeeze()
        preds = (outputs >= 0.5).int()
        all_preds.extend(preds.numpy())

test_acc = accuracy_score(y_test, all_preds)
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.9221


In [ ]:
# Compute average stats per team (same preprocessing as training)
team_stats = df.drop(columns=['Region', 'Result'])
team_averages = team_stats.groupby('Team').mean().reset_index()

# Load data
average_df = pd.read_csv("../datasets/vct_average.csv")  # replace with your file path
average_df = average_df.drop(columns=['Unnamed: 0'])

In [38]:
def predict_average_match(teamA, teamB, average_df, model, scaler):
    # Columns used in training
    columns_to_average = ['Pistol Rounds Won', 'First Kills', 'KAST', 'Clutches', 
                          'Eco', 'Semi-Eco', 'Half-Buy', 'Full-Buy',
                          '+1', '-1', '0', '+2', '-2', '-3', '+3', '+4', '-4']
    columns_to_average_opp = [col + ' Opp' for col in columns_to_average]

    # Get team average stats
    team_rows_1 = average_df[average_df['Team'] == teamA]
    team_rows_2 = average_df[average_df['Team'] == teamB]

    if team_rows_1.empty or team_rows_2.empty:
        raise ValueError("One of the teams is not found in the average DataFrame")

    mean_values_1 = team_rows_1[columns_to_average].mean()
    mean_values_2 = team_rows_2[columns_to_average].mean()

    # Create input rows
    input_1 = mean_values_1.tolist() + mean_values_2.tolist()  # Team A vs B
    input_2 = mean_values_2.tolist() + mean_values_1.tolist()  # Team B vs A

    # Scale the inputs
    scaled_1 = scaler.transform([input_1])
    scaled_2 = scaler.transform([input_2])

    input_tensor_1 = torch.tensor(scaled_1, dtype=torch.float32)
    input_tensor_2 = torch.tensor(scaled_2, dtype=torch.float32)

    # Predict
    model.eval()
    with torch.no_grad():
        prob_1 = model(input_tensor_1).item()  # Probability Team A wins
        prob_2 = model(input_tensor_2).item()  # Probability Team B wins

    print(f"{teamA} vs {teamB}")
    print(f"→ {teamA} win probability: {prob_1:.2f}")
    print(f"→ {teamB} win probability: {prob_2:.2f}")

    if prob_1 > prob_2:
        winner = teamA
    else:
        winner = teamB

    print(f"🧠 Predicted Winner: {winner}")
    return winner, prob_1, prob_2


In [50]:
teamA = "2G"
teamB = "DFM"
winner, teamA_prob, teamB_prob = predict_average_match(teamA, teamB, average_df, model, scaler)

2G vs DFM
→ 2G win probability: 0.27
→ DFM win probability: 0.90
🧠 Predicted Winner: DFM


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [53]:
import joblib

# Save
torch.save(model.state_dict(), "../models/nn_model.pth")

joblib.dump(scaler, "../models/nn_scaler.pkl")

['../models/nn_scaler.pkl']